**Necessary Imports to run before running program**

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

# **Data Processsing for LSUN Dataset**
Run this before running the algorithms / models

In [ ]:
!pip install tensorflow-datasets

In [ ]:
# !pip install tensorflow-io[tensorflow] tensorflow-datasets --no-cache-dir
!pip install tensorflow-io

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 11.9 MB/s eta 0:00:00


In [2]:
dataset_name = 'lsun/classroom'
# train_ds = tfds.load(dataset_name, split='train')
train_ds, val_ds = tfds.load(dataset_name, split=['train', 'validation'], as_supervised=True)
# train_ds, val_ds = tfds.load(dataset_name, split=["train", "validation"])

output_dir = "/content/lsun_dataset/"
os.makedirs(output_dir, exist_ok=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

KeyboardInterrupt: 

In [ ]:
train_ds, test_ds = tfds.load('mnist')

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
# Optional to run
for example in train_ds.take(10):
    image = example['image']
    print("Image shape:", image.shape)

    # Display the image using Matplotlib
    plt.imshow(image.numpy())
    plt.axis('off')
    plt.show()

# **Canny Edge Detection Algorithm**

**Selecting and displaying the image to detect edges from**

In [ ]:
# Method to get the correct image from specified index
def get_image_by_index(ds, index):
    for i, example in enumerate(ds):
        if i == index:
            return example['image'].numpy()
    raise IndexError("Index out of range")

In [ ]:
# Testing  the correct image from specified index
index = 5382 # Can be changed
image = get_image_by_index(train_ds, index)

plt.imshow(image)
plt.axis('off')
plt.show()

**Using Canny to detect edges and display**

In [ ]:
def convert_and_show(image):

  edges = cv.Canny(image, 100, 200)

  # Display the result using Matplotlib
  plt.figure(figsize=(10, 5))

  # Showing the Original Image
  plt.subplot(1, 2, 1)
  plt.title("Original Image")
  plt.imshow(image)
  plt.axis('off')

  # Show the Canny edges
  plt.subplot(1, 2, 2)
  plt.title("Canny Edges")
  plt.imshow(edges, cmap='gray')
  plt.axis('off')

  plt.show() # Displaying

**RGB Image to Canny**

In [ ]:
convert_and_show(image)

**Grayscale Image to Canny**

In [ ]:
image_gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
convert_and_show(image_gray)

# **Dexined Edge Detection Model**


In [ ]:
!git clone https://github.com/xavysp/DexiNed.git

In [ ]:
%cd ..
%cd content/DexiNed

In [ ]:
!pip install torch torchvision opencv-python numpy matplotlib

In [ ]:
import torch
from model import DexiNed

model = DexiNed().eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(model)

In [ ]:
import torchvision.transforms as transforms

image_resized = cv2.resize(image, (512, 512))

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
input_image = transform(image_resized).unsqueeze(0).to(device)

with torch.no_grad():
    output = model(input_image)

edge_map = output[-1].squeeze().cpu().numpy()
edge_map = (edge_map - edge_map.min()) / (edge_map.max() - edge_map.min())

# Display results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image_resized)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(edge_map, cmap="gray")
plt.title("DexiNed Edge Detection")
plt.axis("off")

plt.show()

# **Convolutional Neural Network Edge Detection Model using LSUN**

In [ ]:
# Already processed data from lsun

def preprocess_image(image):
    image = tf.image.resize(image, (256, 256)) / 255.0
    image = tf.image.convert_image_dtype(image, tf.float32)

    gray = tf.image.rgb_to_grayscale(image)
    gray_np = np.array(gray * 255, dtype=np.uint8)
    gray_np = np.squeeze(gray_np, axis=-1)

    edges = cv.Canny(gray_np, 100, 200)
    edges = tf.convert_to_tensor(edges, dtype=tf.float32) / 255.0
    edges = tf.expand_dims(edges, axis=-1)

    return image, edges

def preprocess_image_tf(image):
    image, edges = tf.py_function(preprocess_image, [image], [tf.float32, tf.float32])
    image.set_shape((256, 256, 3))
    edges.set_shape((256, 256, 1))

    return image, edges

In [ ]:
# Apply the labeling procedure
control_amount = 5000
dataset = train_ds.take(control_amount).map(lambda data: preprocess_image_tf(data['image']))

# batch and shuffle
batch_size = 32
train_dataset = dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_dataset = val_ds.map(lambda data: preprocess_image_tf(data['image']))
val_dataset = val_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

NameError: name 'train_ds' is not defined

In [ ]:
# Another model to achieve the same task (using right now)
def cnn_model(input_shape=(256, 256, 3)):
    inputs = layers.Input(shape=input_shape)

    # Encoder (Downsampling)
    def encoder_block(x, filters, kernel_size=(3, 3), padding='same', activation='relu'):
        x = layers.Conv2D(filters, kernel_size, padding=padding)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation)(x)
        x = layers.Conv2D(filters, kernel_size, padding=padding)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation)(x)
        return x

    # Decoder (Upsampling)
    def decoder_block(x, skip_features, filters, kernel_size=(3, 3), padding='same', activation='relu'):
        x = layers.Conv2DTranspose(filters, kernel_size, strides=(2, 2), padding=padding)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation)(x)
        x = layers.concatenate([x, skip_features])  # Skip connection
        x = layers.Conv2D(filters, kernel_size, padding=padding)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation)(x)
        return x

    # Encoder Path
    e1 = encoder_block(inputs, 32)  # Reduced filters
    p1 = layers.MaxPooling2D((2, 2))(e1)

    e2 = encoder_block(p1, 64)  # Reduced filters
    p2 = layers.MaxPooling2D((2, 2))(e2)

    e3 = encoder_block(p2, 128)  # Reduced filters
    p3 = layers.MaxPooling2D((2, 2))(e3)

    # Bottleneck
    bottleneck = encoder_block(p3, 256)  # Reduced filters

    # Decoder Path
    d1 = decoder_block(bottleneck, e3, 128)  # Reduced filters
    d2 = decoder_block(d1, e2, 64)  # Reduced filters
    d3 = decoder_block(d2, e1, 32)  # Reduced filters

    # Output layer
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same')(d3)

    # Build the model
    model = models.Model(inputs, outputs, name='EdgeDetectionModel')
    # print(model.summary())
    return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

checkpoint_callback = ModelCheckpoint(
    filepath='cnn_trial_3.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

In [ ]:
# initialize model and trian the model
model = cnn_model(input_shape=(256, 256, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2,
    callbacks=[early_stopping_callback, checkpoint_callback])

# Saves the model
model.save('cnn_trial_3.keras')

import pickle
with open('training_history_trial_3.pkl', 'wb') as file:
  pickle.dump(history.history, file)

# Plotting the history
plt.figure(figsize=(10,6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("training and validation accuracy")
plt.xlabel("epoch")
plt.ylabel('accuracy')
plt.legend()
plt.grid(True)
plt.show()

**Testing out CNN**

In [ ]:
from tensorflow.keras.models import load_model
# Use this instead of training the model
# upload the keras file from google drive
model = load_model('cnn_trial_2.keras')

In [ ]:
# Getting the random image
import random

def get_random_image(ds, threshold=1000):
  index = random.randrange(5000, threshold)
  for i, example in enumerate(ds):
        if i == index:
            return example['image'], index
  raise IndexError("Index out of range")

threshold = 8000
rand_image, index = get_random_image(train_ds, threshold)
print(index)

# print(f"Random Image at index {index}")
# plt.imshow(rand_image)
# plt.axis("off")
# plt.show()

In [ ]:
# Processing to predict

def process_prediction_image(image):
  image = tf.image.resize(image, (256, 256)) / 255.0
  image = tf.expand_dims(image, axis=0)
  return image

processed = process_prediction_image(rand_image)

In [ ]:
# Prediction
predicted_edges = model.predict(processed).squeeze()

In [ ]:
# Plotting and displaying predicted edges
def plot_prediction(image, predicted_edges):
  plt.figure(figsize=(10, 5))
  # Plot input image
  plt.subplot(1, 2, 1)
  plt.imshow(image)
  plt.title('Input Image')
  plt.axis('off')

  # Plot predicted edges
  plt.subplot(1, 2, 2)
  plt.imshow(predicted_edges, cmap='gray')
  plt.title('Predicted Edges')
  plt.axis('off')

  plt.show()

plot_prediction(rand_image, predicted_edges)